In [1]:
#import os
#import json
#import datetime as dt
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
#import matplotlib.ticker as ticker

#from matplotlib.gridspec import GridSpec
#from matplotlib.dates import DateFormatter
#from matplotlib import dates
import seaborn as sns

from sklearn import linear_model
#from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
#from sklearn import neural_network
#from sklearn import preprocessing
from sklearn.model_selection import train_test_split
#from sklearn.base import BaseEstimator
#from sklearn.base import ClassifierMixin
#from sklearn.base import clone
#from sklearn.pipeline import _name_estimators
#from sklearn import datasets
from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
#from sklearn.model_selection import cross_val_score
#from itertools import product
#from sklearn.model_selection import GridSearchCV
#from sklearn.ensemble import BaggingClassifier
#from sklearn.ensemble import AdaBoostClassifier

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
# load the data

t2mfile     = '/Users/ryaneagan/MAST638/Project/MAST638_Data/ERA5/t2m_2003_2023.nc'
sstfile     = '/Users/ryaneagan/MAST638/Project/MAST638_Data/ERA5/sst_2003_2023.nc'
dlwfile     = '/Users/ryaneagan/MAST638/Project/MAST638_Data/ERA5/dlw_2003_2023.nc'
windfile    = '/Users/ryaneagan/MAST638/Project/MAST638_Data/ERA5/wnd_2003_2023.nc'
cloudfile   = '/Users/ryaneagan/MAST638/Project/MAST638_Data/ERA5/cld_2003_2023.nc'
rhfile      = '/Users/ryaneagan/MAST638/Project/MAST638_Data/ERA5/rhp_2003_2023.nc'
seaicefile  = '/Users/ryaneagan/MAST638/Project/MAST638_Data/seaice_2003_2023.nc'

t2m_raw     = xr.open_dataset(t2mfile,decode_coords="all")
sst_raw     = xr.open_dataset(sstfile,decode_coords="all")
dlw_raw     = xr.open_dataset(dlwfile,decode_coords="all")
wind_raw    = xr.open_dataset(windfile,decode_coords="all")
cloud_raw   = xr.open_dataset(cloudfile,decode_coords="all")
rh_raw      = xr.open_dataset(rhfile,decode_coords="all")
seaice_raw  = xr.open_dataset(seaicefile,decode_coords="all")

In [3]:
# drop all the Feb 29th in leap years

t2m_raw     = t2m_raw.sel(time=~((t2m_raw.time.dt.month == 2) & (t2m_raw.time.dt.day == 29)))
sst_raw     = sst_raw.sel(time=~((sst_raw.time.dt.month == 2) & (sst_raw.time.dt.day == 29)))
dlw_raw     = dlw_raw.sel(time=~((dlw_raw.time.dt.month == 2) & (dlw_raw.time.dt.day == 29)))
wind_raw    = wind_raw.sel(time=~((wind_raw.time.dt.month == 2) & (wind_raw.time.dt.day == 29)))
cloud_raw   = cloud_raw.sel(time=~((cloud_raw.time.dt.month == 2) & (cloud_raw.time.dt.day == 29)))
rh_raw      = rh_raw.sel(time=~((rh_raw.time.dt.month == 2) & (rh_raw.time.dt.day == 29)))
seaice_raw  = seaice_raw.sel(time=~((seaice_raw.time.dt.month == 2) & (seaice_raw.time.dt.day == 29)))

In [4]:
# calculate the wind magnitude and wind direction

wind_raw['magnitude'] = np.sqrt(np.square(wind_raw['u10']) + np.square(wind_raw['v10']))
wind_raw['direction'] = np.rad2deg(np.arctan2(wind_raw['v10'],wind_raw['u10']))

In [5]:
# weith the grid cells 
weights = np.cos(np.deg2rad(t2m_raw.latitude))
weights.name='weights'

In [6]:
# reproject the seaice data from the ease gride to WGS 1984 / epsg:4326 to match ERA 5 data
seaice_reproj = seaice_raw.rio.reproject('epsg:4326')


In [7]:
# weight the cells for the ice data 

iweights = np.cos(np.deg2rad(seaice_reproj.y))
iweights.name='weights'

In [8]:
# get the DDU area form the sea ice file and pull out only the mid concentation
# smith area is 120E to 160E
# DDU 139E, 146E

seaice_area_mask = seaice_reproj.sel(y=slice(-60.0,-70.0),
                                           x=slice(139.0,146.0))



In [9]:
    
seaice_1d = seaice_area_mask['tc_mid'].where((seaice_area_mask['tc_mid'] <= 100.)).weighted(iweights).mean(dim=['x','y']).to_dataframe()

In [10]:
seaice_1d.drop(columns=['lambert_azimuthal_equal_area'],inplace=True)

In [11]:
# reduce spatial dimension

t2m_1d      = t2m_raw.weighted(weights).mean(dim=['longitude','latitude']).to_dataframe()
sst_1d      = sst_raw.weighted(weights).mean(dim=['longitude','latitude']).to_dataframe()
dlw_1d      = dlw_raw.weighted(weights).mean(dim=['longitude','latitude']).to_dataframe()
wind_1d     = wind_raw.weighted(weights).mean(dim=['longitude','latitude']).to_dataframe()
cloud_1d    = cloud_raw.weighted(weights).mean(dim=['longitude','latitude']).to_dataframe()
rh_1d       = rh_raw.weighted(weights).mean(dim=['longitude','latitude']).to_dataframe()


In [12]:
# reduce temporal to weekly means

t2m_weekly      = t2m_1d.resample('W').mean()
sst_weekly      = sst_1d.resample('W').mean()
dlw_weekly      = dlw_1d.resample('W').mean()
wind_weekly     = wind_1d.resample('W').mean()
cloud_weekly    = cloud_1d.resample('W').mean()
rh_weekly       = rh_1d.resample('W').mean()
seaice_weekly   = seaice_1d.resample('W').mean()

In [13]:
# save the processed data for easier loading/processing later
t2mfile_csv     = '/Users/ryaneagan/MAST638/Project/MAST638_Data/CSV/ddu_t2m_2003_2023.csv'
sstfile_csv     = '/Users/ryaneagan/MAST638/Project/MAST638_Data/CSV/ddu_sst_2003_2023.csv'
dlwfile_csv     = '/Users/ryaneagan/MAST638/Project/MAST638_Data/CSV/ddu_dlw_2003_2023.csv'
windfile_csv    = '/Users/ryaneagan/MAST638/Project/MAST638_Data/CSV/ddu_wnd_2003_2023.csv'
cloudfile_csv   = '/Users/ryaneagan/MAST638/Project/MAST638_Data/CSV/ddu_cld_2003_2023.csv'
rhfile_csv      = '/Users/ryaneagan/MAST638/Project/MAST638_Data/CSV/ddu_rhp_2003_2023.csv'
seaicefile_csv  = '/Users/ryaneagan/MAST638/Project/MAST638_Data/CSV/ddu_seaice_2003_2023.csv'

In [14]:
t2m_weekly.to_csv(t2mfile_csv)
sst_weekly.to_csv(sstfile_csv)
dlw_weekly.to_csv(dlwfile_csv)
wind_weekly.to_csv(windfile_csv)
cloud_weekly.to_csv(cloudfile_csv)
rh_weekly.to_csv(rhfile_csv)
seaice_weekly.to_csv(seaicefile_csv)

In [ ]:
def loadcsv(filename):
    df = pd.read_csv(filename)
    df['dateandtime'] = pd.to_datetime(df['time'])
    df.set_index(['dateandtime'],inplace=True)
    df.drop(columns=['time'],inplace=True)
    return df

In [ ]:
# load the files from CSV

t2m_weekly = loadcsv(t2mfile_csv)
sst_weekly = loadcsv(sstfile_csv)
dlw_weekly = loadcsv(dlwfile_csv)
wind_weekly = loadcsv(windfile_csv)
cloud_weekly = loadcsv(cloudfile_csv)
rh_weekly = loadcsv(rhfile_csv)
seaice_weekly = loadcsv(seaicefile_csv)

In [ ]:
# add weeks column for sorting and selecting later

winter_weeks = [45,46,47,48,49,50,51,52,1,2,3,4,5,6,7,8]

t2m_weekly['Week']      = t2m_weekly.index.isocalendar().week
sst_weekly['Week']      = sst_weekly.index.isocalendar().week
dlw_weekly['Week']      = dlw_weekly.index.isocalendar().week
wind_weekly['Week']     = wind_weekly.index.isocalendar().week
cloud_weekly['Week']    = cloud_weekly.index.isocalendar().week
rh_weekly['Week']       = rh_weekly.index.isocalendar().week
seaice_weekly['Week']   = seaice_weekly.index.isocalendar().week


In [ ]:
t2m_winter      = t2m_weekly[t2m_weekly['Week'].isin(winter_weeks)]
sst_winter      = sst_weekly[sst_weekly['Week'].isin(winter_weeks)]
dlw_winter      = dlw_weekly[dlw_weekly['Week'].isin(winter_weeks)]
wind_winter     = wind_weekly[wind_weekly['Week'].isin(winter_weeks)]
cloud_winter    = cloud_weekly[cloud_weekly['Week'].isin(winter_weeks)]
rh_winter       = rh_weekly[rh_weekly['Week'].isin(winter_weeks)]
seaice_winter   = seaice_weekly[seaice_weekly['Week'].isin(winter_weeks)]

t2m_weekly.drop(columns=['Week'],inplace=True)
sst_weekly.drop(columns=['Week'],inplace=True)
dlw_weekly.drop(columns=['Week'],inplace=True)
wind_weekly.drop(columns=['Week'],inplace=True)
cloud_weekly.drop(columns=['Week'],inplace=True)
rh_weekly.drop(columns=['Week'],inplace=True)
seaice_weekly.drop(columns=['Week'],inplace=True)

In [ ]:
t2m_winter.drop(columns=['Week'],inplace=True)
sst_winter.drop(columns=['Week'],inplace=True)
dlw_winter.drop(columns=['Week'],inplace=True)
wind_winter.drop(columns=['Week'],inplace=True)
cloud_winter.drop(columns=['Week'],inplace=True)
rh_winter.drop(columns=['Week'],inplace=True)
seaice_winter.drop(columns=['Week'],inplace=True)

In [ ]:
# drop outlier function

def drop_outliers(df,varName,qmin,qmax):
    s_qmin_max = df[varName].quantile([qmin,qmax])
    newdf = df[(df[varName] > s_qmin_max[qmin]) & (df[varName] < s_qmin_max[qmax])]
    return newdf

In [ ]:
def combineDFS(dfs):
    pdflist = dfs
    from functools import reduce
    df_merged = reduce(lambda  left,right: 
                       pd.merge(left,right,left_index=True,right_index=True,
                                how='outer'), 
                       pdflist)
    
    return df_merged

In [ ]:
t2m_winter = drop_outliers(t2m_winter,'t2m',0.05,0.95)
seaice_winter = drop_outliers(seaice_winter,'tc_mid',0.05,0.95)

In [ ]:
# combine all of the individual dataframes into one dataframe for feature engineering

dfs = [
    t2m_weekly, 
    #sst_weekly,  
    dlw_weekly,  
    wind_weekly,  
    cloud_weekly, 
    rh_weekly,
    seaice_weekly
]

df = combineDFS(dfs)

winter_dfs = [
    t2m_winter,
    #sst_winter,   
    dlw_winter,   
    wind_winter,   
    cloud_winter,  
    rh_winter, 
    seaice_winter 
]

wdf = combineDFS(winter_dfs)

In [ ]:
df = df.reset_index(drop=True)
wdf = wdf.reset_index(drop=True)

In [ ]:
#drop the columns we don't want
#df.drop(columns=['u10','v10','z'],inplace=True)
#wdf.drop(columns=['u10','v10','z'],inplace=True)

In [ ]:
# drop rows with NAN

df = df.dropna(axis=0)
wdf = wdf.dropna(axis=0)

In [ ]:

# add lagged ice cocentration columns
df['tc_mid_1w'] = df['tc_mid'].shift(1)
df['tc_mid_2w'] = df['tc_mid'].shift(2)
df['tc_mid_4w'] = df['tc_mid'].shift(4)

#drop the NAN rows created by the shift
df = df.dropna(axis=0)

In [ ]:
# add lagged ice cocentration columns
wdf['tc_mid_1w'] = wdf['tc_mid'].shift(1)
wdf['tc_mid_2w'] = wdf['tc_mid'].shift(2)
wdf['tc_mid_4w'] = wdf['tc_mid'].shift(4)

#drop the NAN rows created by the shift
wdf = wdf.dropna(axis=0)

In [ ]:
wdf.columns

In [ ]:
from scipy.stats import zscore
zwdf = wdf.apply(zscore)
bxax = sns.boxplot(data=zwdf,color='steelblue')
bxax.set_xticklabels(bxax.get_xticklabels(),rotation=30)
bxax.set(title='Z-Score Feature Box Plot')

In [ ]:
#sns.pairplot(zwdf[['t2m','sst','direction','magnitude','lcc','ssrd','r','tc_mid']])
sns.pairplot(zwdf[['t2m','u10','v10','direction','magnitude','lcc','ssrd','r','tc_mid']])

In [ ]:
#corr = zwdf[['t2m','sst','direction','magnitude','lcc','ssrd','r','tc_mid']].corr()
#corr = zwdf[['t2m','u10','v10','direction','magnitude','lcc','ssrd','r','tc_mid']].corr()
corr = zwdf[['t2m','u10','v10','direction','magnitude','lcc','ssrd','r','tc_mid_1w']].corr()
sns.heatmap(corr, cmap = "GnBu", annot = True)

In [ ]:
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(color='#f1f1f1')  # Color NaNs grey
 .format(precision=2))

In [ ]:
def regression_plot(y_predicted, y_test, title):
    fig, ax = plt.subplots()
    ax.scatter(y_predicted, y_test, edgecolors=(0, 0, 1))
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=3)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title(title)
    fig.show()

In [ ]:
def add_stats(y_predicted, y_test):
    # model evaluation for testing set

    mae = metrics.mean_absolute_error(y_test, y_predicted)
    mse = metrics.mean_squared_error(y_test, y_predicted)
    r2 = metrics.r2_score(y_test, y_predicted)
    evar = metrics.explained_variance_score(y_test, y_predicted)

    print("The model performance for testing set")
    print("--------------------------------------")
    print('MAE is {}'.format(mae))
    print('MSE is {}'.format(mse))
    print('R2 score is {}'.format(r2))
    print('Explained Variance Score is {}'.format(evar))
    print('---------------------------------------"\n')
    
    return [mae,mse,r2,evar]

In [ ]:
model_stats = {'columns':['MAE','MSE','R2','ExpVarScore']}

In [ ]:
alpha_range = [1.0,0.1,0.001,0.0001,0.00001]

In [ ]:
#predictor_cols = ['t2m','sst','direction','magnitude','lcc','ssrd','r']
predictor_cols = ['t2m','u10','v10','direction','magnitude','lcc','ssrd','r']
predictand_col = ['tc_mid_1w']

X_train, X_test, y_train, y_test = train_test_split(wdf[predictor_cols], wdf[predictand_col],test_size=0.3,random_state=5,shuffle=False)

# trial OLS


pipe_linear = Pipeline(
    steps=[
        ('scaler',StandardScaler()),
        ('ols',linear_model.LinearRegression()),

    ]
)
pipe_linear.fit(X_train, y_train)
y_pred = pipe_linear.predict(X_test)

model = pipe_linear.named_steps['ols']
coef = model.coef_
intercept = model.intercept_

sns.barplot(x = predictor_cols, y=coef[0], palette='GnBu').set(title='OLS Feature Importance')

regression_plot(y_pred,y_test,'OLS Regression')

ols_stats = add_stats(y_pred,y_test)
model_stats['ols'] = ols_stats


In [ ]:
pipe_elastic = Pipeline(
    steps=[
        ('scaler',StandardScaler()),
        ('elastic',linear_model.ElasticNet(alpha=1.0, l1_ratio=0.50))

    ]
)

pipe_elastic.fit(X_train, y_train)
y_elastic_pred = pipe_elastic.predict(X_test)

elastic_model = pipe_elastic.named_steps['elastic']

elastic_coef = elastic_model.coef_
elastic_intercept = elastic_model.intercept_

sns.barplot(x = predictor_cols, y=elastic_coef, palette='GnBu').set(title='ElasticNet Feature Importance')

regression_plot(y_elastic_pred,y_test,'Elastic')
elastic_stats = add_stats(y_elastic_pred,y_test)
model_stats['elastic'] = elastic_stats

In [ ]:
tree_model = ensemble.RandomForestRegressor()

tree_model.fit(X_train, y_train)
y_tree_pred = tree_model.predict(X_test)

forest_importances = tree_model.feature_importances_

sns.barplot(x = predictor_cols, y=forest_importances, palette='GnBu').set(title='Random Forest Feature Importance')

regression_plot(y_tree_pred,y_test,'Forest')

forest_stats = add_stats(y_tree_pred,y_test)
model_stats['forest'] = forest_stats

In [ ]:
pipe_ridgecv = Pipeline(
    steps=[
        ('scaler',StandardScaler()),
        ('ridgecv',linear_model.RidgeCV(alphas=alpha_range,cv=5))

    ]
)

pipe_ridgecv.fit(X_train,y_train)

y_ridgecv_pred = pipe_ridgecv.predict(X_test)

ridgecv_model = pipe_ridgecv.named_steps['ridgecv']

ridgecv_alpha = ridgecv_model.alpha_
ridgecv_coef = ridgecv_model.coef_

sns.barplot(x = predictor_cols, y=ridgecv_coef[0], palette='GnBu').set(title='RidgeCV Feature Importance')

regression_plot(y_ridgecv_pred,y_test,'RidgeCV')
ridgecv_stats = add_stats(y_ridgecv_pred,y_test)
model_stats['ridgecv'] = ridgecv_stats

In [ ]:
pipe_lassocv = Pipeline(
    steps=[
        ('scaler',StandardScaler()),
        ('lassocv',linear_model.LassoCV(alphas=alpha_range,cv=5))
    ]
)

pipe_lassocv.fit(X_train,y_train)

lassocv_model = pipe_lassocv.named_steps['lassocv']

y_lassocv_pred = pipe_lassocv.predict(X_test)

lassocv_coef = lassocv_model.coef_
lassocv_alpha = lassocv_model.alpha_

sns.barplot(x = predictor_cols, y=lassocv_coef, palette='GnBu').set(title='LassoCV Feature Importance')
regression_plot(y_lassocv_pred,y_test,'LassoCV')

lassocv_stats = add_stats(y_lassocv_pred,y_test)
model_stats['lassocv'] = lassocv_stats

In [ ]:
pipe_elasticcv = Pipeline(
    steps=[
        ('scaler',StandardScaler()),
        ('elasticcv',linear_model.ElasticNetCV(alphas=alpha_range))
    ]
)

pipe_elasticcv.fit(X_train,y_train)

elasticcv_model = pipe_elasticcv.named_steps['elasticcv']

y_elasticcv_pred = pipe_elasticcv.predict(X_test)

elasticcv_alpha = elasticcv_model.alpha_
elasticcv_coef = elasticcv_model.coef_

sns.barplot(x = predictor_cols, y=elasticcv_coef, palette='GnBu').set(title='ElasticNetCV Feature Importance')
regression_plot(y_elasticcv_pred,y_test,'ElasticNetCV')

elasticcv_stats = add_stats(y_elasticcv_pred,y_test)
model_stats['elasticcv'] = elasticcv_stats

In [ ]:
pipe_bayes = Pipeline(
    steps=[
        ('scaler',StandardScaler()),
        ('bayes',linear_model.BayesianRidge())

    ]
)

pipe_bayes.fit(X_train,y_train)

y_bayes_pred = pipe_bayes.predict(X_test)

bayes_model = pipe_bayes.named_steps['bayes']

bayes_coef = bayes_model.coef_

sns.barplot(x = predictor_cols, y=bayes_coef, palette='GnBu').set(title='Bayes Feature Importance')

regression_plot(y_bayes_pred,y_test,'Bayes')

bayes_stats = add_stats(y_bayes_pred,y_test)
model_stats['bayes'] = bayes_stats

In [ ]:
pipe_sgd = Pipeline(
    steps=[
        ('scaler',StandardScaler()),
        ('sgd',linear_model.SGDRegressor())

    ]
)

pipe_sgd.fit(X_train,y_train)

y_sgd_pred = pipe_sgd.predict(X_test)

sgd_model = pipe_sgd.named_steps['sgd']

sgd_coef = sgd_model.coef_

ax = sns.barplot(x = predictor_cols, y=sgd_coef, palette='GnBu')
ax.set(title='SGD Feature Importance')
for i in ax.containers:
    ax.bar_label(i,)

regression_plot(y_sgd_pred,y_test,'SGD Regression')

sgd_stats = add_stats(y_sgd_pred,y_test)
model_stats['sgd'] = sgd_stats

In [ ]:
barfig, baxes = plt.subplots(4,2,figsize=(20,24))

all_ax = baxes.ravel()

sns.barplot(x = predictor_cols, y=coef[0], palette='GnBu',ax=all_ax[0])
all_ax[0].set(title='OLS Feature Importance')

sns.barplot(x = predictor_cols, y=forest_importances, palette='GnBu',ax=all_ax[1])
all_ax[1].set(title='Forest Feature Importance')

sns.barplot(x = predictor_cols, y=ridgecv_coef[0], palette='GnBu',ax=all_ax[2])
all_ax[2].set(title='RidgeCV Feature Importance')

sns.barplot(x = predictor_cols, y=lassocv_coef, palette='GnBu',ax=all_ax[3])
all_ax[3].set(title='LassoCV Feature Importance')

sns.barplot(x = predictor_cols, y=elastic_coef, palette='GnBu',ax=all_ax[4])
all_ax[4].set(title='Elastic Feature Importance')

sns.barplot(x = predictor_cols, y=elasticcv_coef, palette='GnBu',ax=all_ax[5])
all_ax[5].set(title='ElasticCV Feature Importance')

sns.barplot(x = predictor_cols, y=bayes_coef, palette='GnBu',ax=all_ax[6])
all_ax[6].set(title='Bayes Feature Importance')

sns.barplot(x = predictor_cols, y=sgd_coef, palette='GnBu',ax=all_ax[7])
all_ax[7].set(title='SGD Feature Importance')

for j in range(len(all_ax)):
    for i in all_ax[j].containers:
        all_ax[j].bar_label(i,)

barfig.tight_layout()

In [ ]:
print(model_stats)

In [ ]:
ms = pd.DataFrame.from_dict(model_stats,orient='index')

In [ ]:
ms.rename(columns={0:'MAE',1:'MSE',2:'R2',3:'EVS'},inplace=True)

ms['MAE'] = pd.to_numeric(ms['MAE'], errors='coerce')
ms['MSE'] = pd.to_numeric(ms['MSE'], errors='coerce')
ms['R2'] = pd.to_numeric(ms['R2'], errors='coerce')
ms['EVS'] = pd.to_numeric(ms['EVS'], errors='coerce')

ms.reset_index(drop=True)
ms.drop(index=ms.index[0], axis=0, inplace=True)

In [ ]:
ms.describe()

In [ ]:
model_stats_csv  = '/Users/ryaneagan/MAST638/Project/MAST638_Data/CSV/model_stats.csv'
ms.to_csv(model_stats_csv)

In [ ]:
ms

In [ ]:
print(X_train.count())
print(X_test.count())


In [ ]:
lassocv_alpha

In [ ]:
#tree.plot_tree(tree_model, max_depth=6)

In [ ]:
from sklearn.inspection import permutation_importance


def plot_permutation_importance(clf, X, y, ax):
    result = permutation_importance(clf, X, y, n_repeats=10, random_state=42, n_jobs=2)
    perm_sorted_idx = result.importances_mean.argsort()

    ax.boxplot(
        result.importances[perm_sorted_idx].T,
        vert=False,
        labels=X.columns[perm_sorted_idx],
    )
    ax.axvline(x=0, color="k", linestyle="--")
    return ax

Reference:

https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance_multicollinear.html

https://towardsdatascience.com/feature-selection-using-random-forest-26d7b747597f

https://medium.com/@hertan06/which-features-to-use-in-your-model-350630a1e31c



In [ ]:
# permutation feature importance for random forest

mdi_importances = pd.Series(tree_model.feature_importances_, index=X_train.columns)
tree_importance_sorted_idx = np.argsort(tree_model.feature_importances_)
tree_indices = np.arange(0, len(tree_model.feature_importances_)) + 0.5

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
mdi_importances.sort_values().plot.barh(ax=ax1)
ax1.set_xlabel("Gini importance")
plot_permutation_importance(tree_model, X_train, y_train, ax2)
ax2.set_xlabel("Decrease in accuracy score")
fig.suptitle(
    "Impurity-based vs. permutation importances on multicollinear features (train set)"
)
_ = fig.tight_layout()



In [ ]:
#poly regression

from sklearn.preprocessing import PolynomialFeatures 
degree = 2 
polynomial_features = PolynomialFeatures(degree = degree)

x_train_poly = polynomial_features.fit_transform(X_train)
x_test_poly = polynomial_features.fit_transform(X_test) 
print(x_train_poly)

In [ ]:
pmodel = linear_model.LinearRegression() 
pmodel.fit(x_train_poly, y_train) 
y_poly_pred = model.predict(x_train_poly) 

#---plot the points--
plt.scatter(X_train, y_train, s=10) 
#---plot the regression line--
plt.plot(X_train, y_poly_pred) 
plt.show()